In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
def df_origen(origen,destino):     
    
    ######################PICKLE 21###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names2.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)

    df = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Referencia2.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)    
    
    ######################PICKLE 2###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento1.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df2 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento1.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    #Filtramos el destino
    columns_names_df = df.columns   
    names_1 = list(columns_names_df)     
    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
            
    df_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Referencia2.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)    
    df2_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento1.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)

    
    #Cambiamos la frecuencia a dias
    df.index.freq = df.index.inferred_freq
    df_Dia = df.resample('24H').sum()
    
    df2.index.freq = df2.index.inferred_freq
    df2_Dia = df2.resample('24H').sum()

    #Sacamos una lista de los trayectos origen en comun con mayor número de viajes
    mean_df = df_Dia.mean()
    mean_dict = mean_df.to_dict()
    main_lines = list(dict.keys(dict(filter(lambda x: x[1] > 10000, mean_dict.items())))) #Obtengo las claves de las columnas con un promedio por dia mayor a 8000        
    
    mean_df2 = df2_Dia.mean()
    mean_dict2 = mean_df2.to_dict()
    main_lines2 = list(dict.keys(dict(filter(lambda x: x[1] > 4000, mean_dict2.items())))) #Obtengo las claves de las columnas con un promedio por dia mayor a 8000        
    
    #Sacamos solo los trayectos principales 
    main_lines.insert(0,"ds")    
    df_main_lines = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Referencia2.csv", sep=',',index_col=0, parse_dates=True, usecols=main_lines)
    
    main_lines2.insert(0,"ds")    
    df_main_lines2 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento1.csv", sep=',',index_col=0, parse_dates=True, usecols=main_lines2)
    
    #Añadimos nuevas columnas
    df_main_lines['Fecha'] = df_main_lines.index.date
    df_main_lines['Año'] = df_main_lines.index.year                                
    df_main_lines['Mes'] = df_main_lines.index.month                               
    df_main_lines['ID_Dia'] = df_main_lines.index.dayofweek
    df_main_lines['Hora'] = df_main_lines.index.hour
    
    df_main_lines2['Fecha'] = df_main_lines2.index.date
    df_main_lines2['Año'] = df_main_lines2.index.year                                
    df_main_lines2['Mes'] = df_main_lines2.index.month                               
    df_main_lines2['ID_Dia'] = df_main_lines2.index.dayofweek
    df_main_lines2['Hora'] = df_main_lines2.index.hour
    
    df_Final['Fecha'] = df_Final.index.date
    df_Final['Año'] = df_Final.index.year                                
    df_Final['Mes'] = df_Final.index.month                               
    df_Final['ID_Dia'] = df_Final.index.dayofweek
    df_Final['Hora'] = df_Final.index.hour
    df_Final['WeekNo'] = df_Final.index.isocalendar().week
    
    df2_Final['Fecha'] = df2_Final.index.date
    df2_Final['Año'] = df2_Final.index.year                                
    df2_Final['Mes'] = df2_Final.index.month                               
    df2_Final['ID_Dia'] = df2_Final.index.dayofweek
    df2_Final['Hora'] = df2_Final.index.hour
    df2_Final['WeekNo'] = df2_Final.index.isocalendar().week
    
    return df, lista, df_Dia, main_lines, df_main_lines, df2, df2_Dia, main_lines2, df_main_lines2, df_Final,df2_Final,listaDst

In [3]:
origen = "2807913"
destino = "2807903"
df,lista, df_Dia, main_lines, df_main_lines, df2, df2_Dia, main_lines2, df_main_lines2,df_Final,df2_Final,listaDst = df_origen(origen,destino)

In [4]:
df_main_lines.head(5)   ##Principales trayectos en el periodo de referencia
df_main_lines2.head(5)  #Principales trayectos en el periodo de confinamiento
df_Final.head(5)  #Trayecto en el periodo de referencia
df2_Final.head(5) #Trayecto en el Periodo confinamiento

,2807913-2807903,Fecha,Año,Mes,ID_Dia,Hora,WeekNo
ds,,,,,,,
2020-03-17 00:00:00,92.242,2020-03-17,2020,3,1,0,12
2020-03-17 01:00:00,164.175,2020-03-17,2020,3,1,1,12
2020-03-17 02:00:00,106.009,2020-03-17,2020,3,1,2,12
2020-03-17 03:00:00,131.052,2020-03-17,2020,3,1,3,12
2020-03-17 04:00:00,187.772,2020-03-17,2020,3,1,4,12


In [5]:
#Creamos intervalos de datos por periodos
data1=df_Final.loc['2020-02-14':'2020-03-15']
data2=df2_Final.loc['2020-03-17':'2020-03-31']
data3=df2_Final.loc['2020-04-01':'2020-04-30']
data4=df2_Final.loc['2020-05-01':'2020-05-31']

In [6]:
##### Creamos tablas solo lunes o solo martes...
lista_periodos=[1,2,3,4]    
dict_dias={0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}
        
for periodo in lista_periodos:        
    cont=0
    globals()[f"d{periodo}"] = globals()[f"data{periodo}"].reset_index()                                
    globals()[f"df_{periodo}"] = pd.DataFrame(data=globals()[f"d{periodo}"], columns=[listaDst[1],'Hora','Fecha','WeekNo','ID_Dia'])
    globals()[f"df_{periodo}"].set_index([listaDst[1]], inplace = True) 
    
    for dia in dict_dias:        
        globals()[f"df_{dict_dias[dia]}_{periodo}"] = globals()[f"df_{periodo}"].loc[globals()[f"df_{periodo}"].loc[:,'ID_Dia'] == cont]  # Seleccionamos solo los datos del lunes o solo los datos del martes para crear nuevas tablas dependiendo del día        
        globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"] = pd.DataFrame({})  #Creamos un nuevo dataframe vacio
        globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"] = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].append(globals()[f"df_{dict_dias[dia]}_{periodo}"])  #Guardamos en el dataframe vacio cada una de las tablas, hay tabla lunes, tabla martes..
        
        #Seleccionamos celdas del dataframe para generar las fechas que se requieren graficar
        start = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].iloc[0]['Fecha'] #Primera celda de la columna fecha
        period_start = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].iloc[0]['WeekNo']
        period_end = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].at[globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].index[-1],'WeekNo']  #última celda de columna WeekNo
        
        #Condición de error en period_start/period_end
        if (type(period_start) == pd.core.series.Series) | (type(period_end) == pd.core.series.Series):    
            lista = list(period_end)
            period_end = lista[-1]        
        
        periods = list(range(period_start, period_end+1))

        #Obtenemos las fechas de cada día 
        date_generated = pd.date_range(start, periods=len(periods), freq='7D')
        
        #Creamos una tabla para cada fecha de cada lunes o de cada martes..
        cont2=1        
        globals()[f"{dict_dias[dia]}_{periodo}"] = pd.DataFrame({})
        for date in date_generated:            
            globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}_{cont2}"] = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].loc[globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].loc[:,'Fecha'] == date]            
            globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}_{cont2}"])
            cont2=cont2+1
        cont=cont+1
    

/tmp/ipykernel_29843/1448273250.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"] = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].append(globals()[f"df_{dict_dias[dia]}_{periodo}"])  #Guardamos en el dataframe vacio cada una de las tablas, hay tabla lunes, tabla martes..
/home/jonathan/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dic

/tmp/ipykernel_29843/1448273250.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"] = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].append(globals()[f"df_{dict_dias[dia]}_{periodo}"])  #Guardamos en el dataframe vacio cada una de las tablas, hay tabla lunes, tabla martes..
/home/jonathan/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dic

/tmp/ipykernel_29843/1448273250.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"] = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].append(globals()[f"df_{dict_dias[dia]}_{periodo}"])  #Guardamos en el dataframe vacio cada una de las tablas, hay tabla lunes, tabla martes..
/home/jonathan/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dic

/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}_{cont2}"])
/tmp/ipykernel_29843/1448273250.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"] = globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}"].append(globals()[f"df_{dict_dias[dia]}_{periodo}"])  #Guardamos en el dataframe vacio cada una de las tablas, hay tabla lunes, tabla martes..
/home/jonathan/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be cons

/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}_{cont2}"])
/home/jonathan/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{d

/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}_{cont2}"])
/home/jonathan/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{d

/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{dict_dias[dia]}_{periodo}_{cont2}"])
/home/jonathan/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/tmp/ipykernel_29843/1448273250.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  globals()[f"{dict_dias[dia]}_{periodo}"]=globals()[f"{dict_dias[dia]}_{periodo}"].append(globals()[f"filtrado_Data_{d

In [7]:
Tuesday_1

,Hora,Fecha,WeekNo,ID_Dia
2807913-2807903,,,,
247.309,0,2020-02-18,8,1
166.532,1,2020-02-18,8,1
155.109,2,2020-02-18,8,1
319.405,3,2020-02-18,8,1
415.364,4,2020-02-18,8,1
...,...,...,...,...
748.007,19,2020-03-10,11,1
504.490,20,2020-03-10,11,1
395.786,21,2020-03-10,11,1


# Clustering

# Periodo de Referencia

In [8]:
Monday_1 = Monday_1.reset_index()
Monday_1 = Monday_1[['Hora','2807913-2807903','Fecha']]
Tuesday_1 = Tuesday_1.reset_index()
Tuesday_1 = Tuesday_1[['Hora','2807913-2807903','Fecha']]
Wednesday_1 = Wednesday_1.reset_index()
Wednesday_1 = Wednesday_1[['Hora','2807913-2807903','Fecha']]
Thursday_1 = Thursday_1.reset_index()
Thursday_1 = Thursday_1[['Hora','2807913-2807903','Fecha']]
Friday_1 = Friday_1.reset_index()
Friday_1 = Friday_1[['Hora','2807913-2807903','Fecha']]

Monday_1['Monday']=Monday_1['2807913-2807903']
Monday_1['Tuesday']=Tuesday_1['2807913-2807903']
Monday_1['Wednesday']=Wednesday_1['2807913-2807903']
Monday_1['Thursday']=Thursday_1['2807913-2807903']
Monday_1['Friday']=Friday_1['2807913-2807903']

PeriodoRef = Monday_1[['Fecha','Monday','Tuesday','Wednesday','Thursday','Friday']]
PeriodoRef.set_index('Fecha',inplace=True)
PeriodoRef.head()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(PeriodoRef.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(PeriodoRef.T)):
    clusters[l].append(PeriodoRef.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print()
    
Table_cluster = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Dias'])

#####################################################
Table_Monday_1 = pd.pivot_table(Monday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Monday_1.columns =[str(s2) for (s1,s2) in Table_Monday_1.columns.tolist()]
Table_Monday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Monday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Monday_1.T)):
    clusters[l].append(Table_Monday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Monday'] = Table_cluster_1['Fecha']
Table_cluster

#####################################################################
Table_Tuesday_1 = pd.pivot_table(Tuesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Tuesday_1.columns =[str(s2) for (s1,s2) in Table_Tuesday_1.columns.tolist()]
Table_Tuesday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Tuesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Tuesday_1.T)):
    clusters[l].append(Table_Tuesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Tuesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Wednesday_1 = pd.pivot_table(Wednesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Wednesday_1.columns =[str(s2) for (s1,s2) in Table_Wednesday_1.columns.tolist()]
Table_Wednesday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Wednesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Wednesday_1.T)):
    clusters[l].append(Table_Wednesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Wednesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Thursday_1 = pd.pivot_table(Thursday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Thursday_1.columns =[str(s2) for (s1,s2) in Table_Thursday_1.columns.tolist()]
Table_Thursday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Thursday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Thursday_1.T)):
    clusters[l].append(Table_Thursday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Thursday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Friday_1 = pd.pivot_table(Friday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Friday_1.columns =[str(s2) for (s1,s2) in Table_Friday_1.columns.tolist()]
Table_Friday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Friday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Friday_1.T)):
    clusters[l].append(Table_Friday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Friday'] = Table_cluster_1['Fecha']
cluster_1 = Table_cluster
cluster_1

Cluster 0: Wednesday
Thursday
Cluster 1: Friday
Cluster 2: Monday
Tuesday
Cluster 0: 2020-03-02
Cluster 1: 2020-02-17
2020-02-24
Cluster 2: 2020-03-09
Cluster 0: 2020-02-18
2020-03-03
Cluster 1: 2020-03-10
Cluster 2: 2020-02-25
Cluster 0: 2020-02-26
2020-03-04
Cluster 1: 2020-03-11
Cluster 2: 2020-02-19
Cluster 0: 2020-03-05
Cluster 1: 2020-03-12
Cluster 2: 2020-02-20
2020-02-27
Cluster 0: 2020-02-14
2020-02-21
2020-03-06
Cluster 1: 2020-03-13
Cluster 2: 2020-02-28


,Cluster,Dias,Monday,Tuesday,Wednesday,Thursday,Friday
0,0,"[Wednesday, Thursday]",[2020-03-02],"[2020-02-18, 2020-03-03]","[2020-02-26, 2020-03-04]",[2020-03-05],"[2020-02-14, 2020-02-21, 2020-03-06]"
1,1,[Friday],"[2020-02-17, 2020-02-24]",[2020-03-10],[2020-03-11],[2020-03-12],[2020-03-13]
2,2,"[Monday, Tuesday]",[2020-03-09],[2020-02-25],[2020-02-19],"[2020-02-20, 2020-02-27]",[2020-02-28]


# Marzo

In [9]:
Monday_1 = Monday_2
Tuesday_1 = Tuesday_2
Wednesday_1 = Wednesday_2
Thursday_1 = Thursday_2
Friday_1 = Friday_2

In [10]:
Monday_1 = Monday_1.reset_index()
Monday_1 = Monday_1[['Hora','2807913-2807903','Fecha']]
Tuesday_1 = Tuesday_1.reset_index()
Tuesday_1 = Tuesday_1[['Hora','2807913-2807903','Fecha']]
Wednesday_1 = Wednesday_1.reset_index()
Wednesday_1 = Wednesday_1[['Hora','2807913-2807903','Fecha']]
Thursday_1 = Thursday_1.reset_index()
Thursday_1 = Thursday_1[['Hora','2807913-2807903','Fecha']]
Friday_1 = Friday_1.reset_index()
Friday_1 = Friday_1[['Hora','2807913-2807903','Fecha']]

Monday_1['Monday']=Monday_1['2807913-2807903']
Monday_1['Tuesday']=Tuesday_1['2807913-2807903']
Monday_1['Wednesday']=Wednesday_1['2807913-2807903']
Monday_1['Thursday']=Thursday_1['2807913-2807903']
Monday_1['Friday']=Friday_1['2807913-2807903']

PeriodoRef = Monday_1[['Fecha','Monday','Tuesday','Wednesday','Thursday','Friday']]
PeriodoRef.set_index('Fecha',inplace=True)
PeriodoRef.head()

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(PeriodoRef.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(PeriodoRef.T)):
    clusters[l].append(PeriodoRef.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print()
    
Table_cluster = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Dias'])

#####################################################
Table_Monday_1 = pd.pivot_table(Monday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Monday_1.columns =[str(s2) for (s1,s2) in Table_Monday_1.columns.tolist()]
Table_Monday_1.tail()

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(Table_Monday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Monday_1.T)):
    clusters[l].append(Table_Monday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Monday'] = Table_cluster_1['Fecha']
Table_cluster

#####################################################################
Table_Tuesday_1 = pd.pivot_table(Tuesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Tuesday_1.columns =[str(s2) for (s1,s2) in Table_Tuesday_1.columns.tolist()]
Table_Tuesday_1.tail()

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(Table_Tuesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Tuesday_1.T)):
    clusters[l].append(Table_Tuesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Tuesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Wednesday_1 = pd.pivot_table(Wednesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Wednesday_1.columns =[str(s2) for (s1,s2) in Table_Wednesday_1.columns.tolist()]
Table_Wednesday_1.tail()

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(Table_Wednesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Wednesday_1.T)):
    clusters[l].append(Table_Wednesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Wednesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Thursday_1 = pd.pivot_table(Thursday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Thursday_1.columns =[str(s2) for (s1,s2) in Table_Thursday_1.columns.tolist()]
Table_Thursday_1.tail()

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(Table_Thursday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Thursday_1.T)):
    clusters[l].append(Table_Thursday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Thursday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Friday_1 = pd.pivot_table(Friday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Friday_1.columns =[str(s2) for (s1,s2) in Table_Friday_1.columns.tolist()]
Table_Friday_1.tail()

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(Table_Friday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Friday_1.T)):
    clusters[l].append(Table_Friday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Friday'] = Table_cluster_1['Fecha']
cluster_2 = Table_cluster
cluster_2

Cluster 0: Monday
Thursday
Friday
Cluster 1: Tuesday
Wednesday
Cluster 0: 2020-03-23
Cluster 1: 2020-03-30
Cluster 0: 2020-03-31
Cluster 1: 2020-03-17
2020-03-24
Cluster 0: 2020-03-18
Cluster 1: 2020-03-25
Cluster 0: 2020-03-19
Cluster 1: 2020-03-26
Cluster 0: 2020-03-20
Cluster 1: 2020-03-27


,Cluster,Dias,Monday,Tuesday,Wednesday,Thursday,Friday
0,0,"[Monday, Thursday, Friday]",[2020-03-23],[2020-03-31],[2020-03-18],[2020-03-19],[2020-03-20]
1,1,"[Tuesday, Wednesday]",[2020-03-30],"[2020-03-17, 2020-03-24]",[2020-03-25],[2020-03-26],[2020-03-27]


# Abril

In [11]:
Monday_1 = Monday_3
Tuesday_1 = Tuesday_3
Wednesday_1 = Wednesday_3
Thursday_1 = Thursday_3
Friday_1 = Friday_3

In [12]:
Monday_1 = Monday_1.reset_index()
Monday_1 = Monday_1[['Hora','2807913-2807903','Fecha']]
Tuesday_1 = Tuesday_1.reset_index()
Tuesday_1 = Tuesday_1[['Hora','2807913-2807903','Fecha']]
Wednesday_1 = Wednesday_1.reset_index()
Wednesday_1 = Wednesday_1[['Hora','2807913-2807903','Fecha']]
Thursday_1 = Thursday_1.reset_index()
Thursday_1 = Thursday_1[['Hora','2807913-2807903','Fecha']]
Friday_1 = Friday_1.reset_index()
Friday_1 = Friday_1[['Hora','2807913-2807903','Fecha']]

Monday_1['Monday']=Monday_1['2807913-2807903']
Monday_1['Tuesday']=Tuesday_1['2807913-2807903']
Monday_1['Wednesday']=Wednesday_1['2807913-2807903']
Monday_1['Thursday']=Thursday_1['2807913-2807903']
Monday_1['Friday']=Friday_1['2807913-2807903']

PeriodoRef = Monday_1[['Fecha','Monday','Tuesday','Wednesday','Thursday','Friday']]
PeriodoRef.set_index('Fecha',inplace=True)
PeriodoRef.head()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(PeriodoRef.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(PeriodoRef.T)):
    clusters[l].append(PeriodoRef.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print()
    
Table_cluster = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Dias'])

#####################################################
Table_Monday_1 = pd.pivot_table(Monday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Monday_1.columns =[str(s2) for (s1,s2) in Table_Monday_1.columns.tolist()]
Table_Monday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Monday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Monday_1.T)):
    clusters[l].append(Table_Monday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Monday'] = Table_cluster_1['Fecha']
Table_cluster

#####################################################################
Table_Tuesday_1 = pd.pivot_table(Tuesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Tuesday_1.columns =[str(s2) for (s1,s2) in Table_Tuesday_1.columns.tolist()]
Table_Tuesday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Tuesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Tuesday_1.T)):
    clusters[l].append(Table_Tuesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Tuesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Wednesday_1 = pd.pivot_table(Wednesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Wednesday_1.columns =[str(s2) for (s1,s2) in Table_Wednesday_1.columns.tolist()]
Table_Wednesday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Wednesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Wednesday_1.T)):
    clusters[l].append(Table_Wednesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Wednesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Thursday_1 = pd.pivot_table(Thursday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Thursday_1.columns =[str(s2) for (s1,s2) in Table_Thursday_1.columns.tolist()]
Table_Thursday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Thursday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Thursday_1.T)):
    clusters[l].append(Table_Thursday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Thursday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Friday_1 = pd.pivot_table(Friday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Friday_1.columns =[str(s2) for (s1,s2) in Table_Friday_1.columns.tolist()]
Table_Friday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Friday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Friday_1.T)):
    clusters[l].append(Table_Friday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Friday'] = Table_cluster_1['Fecha']
cluster_3 = Table_cluster
cluster_3

Cluster 0: Thursday
Friday
Cluster 1: Tuesday
Wednesday
Cluster 2: Monday
Cluster 0: 2020-04-13
2020-04-20
Cluster 1: 2020-04-06
Cluster 2: 2020-04-27
Cluster 0: 2020-04-21
Cluster 1: 2020-04-28
Cluster 2: 2020-04-07
2020-04-14
Cluster 0: 2020-04-22
2020-04-29
Cluster 1: 2020-04-08
2020-04-15
Cluster 2: 2020-04-01
Cluster 0: 2020-04-16
2020-04-23
2020-04-30
Cluster 1: 2020-04-09
Cluster 2: 2020-04-02
Cluster 0: 2020-04-03
2020-04-17
Cluster 1: 2020-04-10
Cluster 2: 2020-04-24


,Cluster,Dias,Monday,Tuesday,Wednesday,Thursday,Friday
0,0,"[Thursday, Friday]","[2020-04-13, 2020-04-20]",[2020-04-21],"[2020-04-22, 2020-04-29]","[2020-04-16, 2020-04-23, 2020-04-30]","[2020-04-03, 2020-04-17]"
1,1,"[Tuesday, Wednesday]",[2020-04-06],[2020-04-28],"[2020-04-08, 2020-04-15]",[2020-04-09],[2020-04-10]
2,2,[Monday],[2020-04-27],"[2020-04-07, 2020-04-14]",[2020-04-01],[2020-04-02],[2020-04-24]


# Mayo

In [13]:
Monday_1 = Monday_4
Tuesday_1 = Tuesday_4
Wednesday_1 = Wednesday_4
Thursday_1 = Thursday_4
Friday_1 = Friday_4

In [14]:
Monday_1 = Monday_1.reset_index()
Monday_1 = Monday_1[['Hora','2807913-2807903','Fecha']]
Tuesday_1 = Tuesday_1.reset_index()
Tuesday_1 = Tuesday_1[['Hora','2807913-2807903','Fecha']]
Wednesday_1 = Wednesday_1.reset_index()
Wednesday_1 = Wednesday_1[['Hora','2807913-2807903','Fecha']]
Thursday_1 = Thursday_1.reset_index()
Thursday_1 = Thursday_1[['Hora','2807913-2807903','Fecha']]
Friday_1 = Friday_1.reset_index()
Friday_1 = Friday_1[['Hora','2807913-2807903','Fecha']]

Monday_1['Monday']=Monday_1['2807913-2807903']
Monday_1['Tuesday']=Tuesday_1['2807913-2807903']
Monday_1['Wednesday']=Wednesday_1['2807913-2807903']
Monday_1['Thursday']=Thursday_1['2807913-2807903']
Monday_1['Friday']=Friday_1['2807913-2807903']

PeriodoRef = Monday_1[['Fecha','Monday','Tuesday','Wednesday','Thursday','Friday']]
PeriodoRef.set_index('Fecha',inplace=True)
PeriodoRef.head()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(PeriodoRef.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(PeriodoRef.T)):
    clusters[l].append(PeriodoRef.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print()
    
Table_cluster = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Dias'])

#####################################################
Table_Monday_1 = pd.pivot_table(Monday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Monday_1.columns =[str(s2) for (s1,s2) in Table_Monday_1.columns.tolist()]
Table_Monday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Monday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Monday_1.T)):
    clusters[l].append(Table_Monday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Monday'] = Table_cluster_1['Fecha']
Table_cluster

#####################################################################
Table_Tuesday_1 = pd.pivot_table(Tuesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Tuesday_1.columns =[str(s2) for (s1,s2) in Table_Tuesday_1.columns.tolist()]
Table_Tuesday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Tuesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Tuesday_1.T)):
    clusters[l].append(Table_Tuesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Tuesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Wednesday_1 = pd.pivot_table(Wednesday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Wednesday_1.columns =[str(s2) for (s1,s2) in Table_Wednesday_1.columns.tolist()]
Table_Wednesday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Wednesday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Wednesday_1.T)):
    clusters[l].append(Table_Wednesday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Wednesday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Thursday_1 = pd.pivot_table(Thursday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Thursday_1.columns =[str(s2) for (s1,s2) in Table_Thursday_1.columns.tolist()]
Table_Thursday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Thursday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Thursday_1.T)):
    clusters[l].append(Table_Thursday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Thursday'] = Table_cluster_1['Fecha']
Table_cluster

############################################################
Table_Friday_1 = pd.pivot_table(Friday_1, values=['2807913-2807903'], index=["Hora"], columns="Fecha", aggfunc={'2807913-2807903': 'first'})
Table_Friday_1.columns =[str(s2) for (s1,s2) in Table_Friday_1.columns.tolist()]
Table_Friday_1.tail()

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(Table_Friday_1.T);

clusters = {}
for l in np.unique(kmeans.labels_):
    clusters[l] = []

for i,l in enumerate(kmeans.predict(Table_Friday_1.T)):
    clusters[l].append(Table_Friday_1.columns[i])    

for c in sorted(clusters):
    print('Cluster ' + str(c) + ': ', end='')
    for symbol in clusters[c]:
        print(symbol)
    #print()
    #print() 
    
Table_cluster_1 = pd.DataFrame([[key, clusters[key]] for key in clusters.keys()], columns=['Cluster', 'Fecha'])
Table_cluster['Friday'] = Table_cluster_1['Fecha']
cluster_4 = Table_cluster
cluster_4

Cluster 0: Tuesday
Thursday
Cluster 1: Friday
Cluster 2: Monday
Wednesday
Cluster 0: 2020-05-18
Cluster 1: 2020-05-25
Cluster 2: 2020-05-04
2020-05-11
Cluster 0: 2020-05-05
2020-05-19
Cluster 1: 2020-05-26
Cluster 2: 2020-05-12
Cluster 0: 2020-05-13
2020-05-20
Cluster 1: 2020-05-27
Cluster 2: 2020-05-06
Cluster 0: 2020-05-21
Cluster 1: 2020-05-28
Cluster 2: 2020-05-07
2020-05-14
Cluster 0: 2020-05-29
Cluster 1: 2020-05-01
2020-05-15
Cluster 2: 2020-05-08
2020-05-22


,Cluster,Dias,Monday,Tuesday,Wednesday,Thursday,Friday
0,0,"[Tuesday, Thursday]",[2020-05-18],"[2020-05-05, 2020-05-19]","[2020-05-13, 2020-05-20]",[2020-05-21],[2020-05-29]
1,1,[Friday],[2020-05-25],[2020-05-26],[2020-05-27],[2020-05-28],"[2020-05-01, 2020-05-15]"
2,2,"[Monday, Wednesday]","[2020-05-04, 2020-05-11]",[2020-05-12],[2020-05-06],"[2020-05-07, 2020-05-14]","[2020-05-08, 2020-05-22]"


In [15]:
pd.concat([cluster_1, cluster_2, cluster_3, cluster_4])

,Cluster,Dias,Monday,Tuesday,Wednesday,Thursday,Friday
0,0,"[Wednesday, Thursday]",[2020-03-02],"[2020-02-18, 2020-03-03]","[2020-02-26, 2020-03-04]",[2020-03-05],"[2020-02-14, 2020-02-21, 2020-03-06]"
1,1,[Friday],"[2020-02-17, 2020-02-24]",[2020-03-10],[2020-03-11],[2020-03-12],[2020-03-13]
2,2,"[Monday, Tuesday]",[2020-03-09],[2020-02-25],[2020-02-19],"[2020-02-20, 2020-02-27]",[2020-02-28]
0,0,"[Monday, Thursday, Friday]",[2020-03-23],[2020-03-31],[2020-03-18],[2020-03-19],[2020-03-20]
1,1,"[Tuesday, Wednesday]",[2020-03-30],"[2020-03-17, 2020-03-24]",[2020-03-25],[2020-03-26],[2020-03-27]
0,0,"[Thursday, Friday]","[2020-04-13, 2020-04-20]",[2020-04-21],"[2020-04-22, 2020-04-29]","[2020-04-16, 2020-04-23, 2020-04-30]","[2020-04-03, 2020-04-17]"
1,1,"[Tuesday, Wednesday]",[2020-04-06],[2020-04-28],"[2020-04-08, 2020-04-15]",[2020-04-09],[2020-04-10]
2,2,[Monday],[2020-04-27],"[2020-04-07, 2020-04-14]",[2020-04-01],[2020-04-02],[2020-04-24]
0,0,"[Tuesday, Thursday]",[2020-05-18],"[2020-05-05, 2020-05-19]","[2020-05-13, 2020-05-20]",[2020-05-21],[2020-05-29]
1,1,[Friday],[2020-05-25],[2020-05-26],[2020-05-27],[2020-05-28],"[2020-05-01, 2020-05-15]"
